In [55]:
import torch
from torchvision.models import resnet18
import pickle
import numpy as np

In [56]:
model = resnet18()

In [135]:
xlayer = model.layer1[1].conv1
scale_factor = 2
kernel_vectors = xlayer.weight.flatten(1)
l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
step_size = (l2_distances.shape[1] // (scale_factor )) + 1
extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
repulsion_loss = -l2_blocks[extract_indices].sum()

In [136]:
l2_blocks[extract_indices].sum(dim=1)

tensor([1.8898, 1.9744, 1.9916, 1.9042, 2.0190, 2.0717, 2.0623, 2.1084, 2.0609,
        1.9098, 2.0160, 1.9300, 2.0273, 2.0526, 2.0465, 2.0106, 1.9866, 2.0052,
        1.9085, 2.0136, 2.0548, 1.9624, 1.9319, 1.9214, 2.0467, 1.9270, 2.0017,
        1.9693, 2.0557, 1.9716, 1.9894, 1.9667], grad_fn=<SumBackward1>)

In [137]:
repulsion_loss

tensor(-63.7877, grad_fn=<NegBackward0>)

In [143]:
i = 0
tot_repulsion_loss = 0
for name, param in model.named_modules():
    if "conv" in name and "layer" in name:
        kernel_vectors = param.weight.flatten(1)
        l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
        l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
        step_size = (l2_distances.shape[1] // (scale_factor )) + 1
        extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
        repulsion_loss = -l2_blocks[extract_indices].sum()
        tot_repulsion_loss+=repulsion_loss    